# Task 2 — Bayesian Change-Point Modeling (PyMC)

Notebook content saved as **task2_bayesian_changepoint.ipynb.json** for download.
Rename locally to **task2_bayesian_changepoint.ipynb**.

## Sections
1. Import + repo path setup (`sys.path`)
2. Load processed log returns + feature checks
3. EDA plots via repo utilities
4. Model 1 (mandatory): mean switch
5. Model 2 (extension): sigma switch
6. Diagnostics + posterior plots
7. Posterior predictive checks (PPC)
8. Impact summaries + tau sample → date mapping + tau-date mass plots
9. Events near change-point (optional)
10. Optional model comparison (LOO)


## 1) Import setup (repo-aware) + plotting defaults

In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import arviz as az
import pymc as pm

# --- Default Font

plt.rcParams['font.family'] = 'sans-serif'  # use system default
plt.rcParams['axes.unicode_minus'] = False  # fixes minus sign display

# --- Make repo root importable ---
_HERE = Path.cwd()
if (_HERE / 'src').exists():
    _ROOT = _HERE
elif (_HERE.parent / 'src').exists():
    _ROOT = _HERE.parent
else:
    _ROOT = _HERE
    for p in _HERE.parents:
        if (p / 'src').exists():
            _ROOT = p
            break

if str(_ROOT) not in sys.path:
    sys.path.insert(0, str(_ROOT))

print('Repo root:', _ROOT)
az.style.use('arviz-darkgrid')
plt.rcParams['figure.dpi'] = 120

## 2) Import project modules (exact repo imports)

In [ ]:
from src.config import (
    ensure_dirs,
    DATA_PROCESSED_DIR,
    DATA_RAW_DIR,
    REPORTS_FIGURES_DIR,
    REPORTS_INTERIM_DIR,
    LOG_RETURNS_FILENAME,
    EVENTS_FILENAME,
    ROLLING_VOL_WINDOW_DAYS,
    EVENT_MATCH_WINDOW_DAYS,
    COL_DATE,
    COL_LOG_RETURN,
)

from src.data.io_task2 import load_log_returns_csv, ensure_log_features
from src.events.io_task2 import load_events
from src.eda.plots_task2 import plot_price, plot_log_returns, plot_rolling_volatility

from src.models.bayes_changepoint_task2 import (
    build_switchpoint_mean_model,
    build_switchpoint_sigma_model,
    sample_model,
    compute_impact_summary,
    compute_sigma_impact_summary,
    map_tau_samples_to_dates,
    compute_convergence_report,
    prior_settings_summary,
    standardize,
)


## 3) Create output directories

In [ ]:
ensure_dirs()
REPORTS_FIGURES_DIR.mkdir(parents=True, exist_ok=True)
REPORTS_INTERIM_DIR.mkdir(parents=True, exist_ok=True)
print('Figures dir:', REPORTS_FIGURES_DIR)
print('Interim dir:', REPORTS_INTERIM_DIR)


## 4) Load processed log returns

In [ ]:
data_path = DATA_PROCESSED_DIR / LOG_RETURNS_FILENAME
df = load_log_returns_csv(data_path)
df = ensure_log_features(df)
df[[COL_DATE, COL_LOG_RETURN]].head()


## 5) EDA figures (via repo plotting utilities)

In [ ]:
# Price series
fig1 = plot_price(df)
fig1.savefig(REPORTS_FIGURES_DIR / 'nb_task2_01_price_raw.png', dpi=150, bbox_inches='tight')
plt.close(fig1)

# Log returns
fig2 = plot_log_returns(df)
fig2.savefig(REPORTS_FIGURES_DIR / 'nb_task2_02_log_returns.png', dpi=150, bbox_inches='tight')
plt.close(fig2)

# Rolling volatility
fig3 = plot_rolling_volatility(df, window=ROLLING_VOL_WINDOW_DAYS)
fig3.savefig(REPORTS_FIGURES_DIR / f'nb_task2_03_rolling_vol_{ROLLING_VOL_WINDOW_DAYS}d.png', dpi=150, bbox_inches='tight')
plt.close(fig3)

print('Saved EDA figures to', REPORTS_FIGURES_DIR)


## 6) Prepare model inputs

In [ ]:
y = df[COL_LOG_RETURN].to_numpy(dtype=float)
mask = np.isfinite(y)
y_clean = y[mask]
dates_clean = df.loc[mask, COL_DATE].reset_index(drop=True)

print('n_clean:', len(y_clean))
print('date range:', dates_clean.iloc[0], '→', dates_clean.iloc[-1])


## 7) (Optional) Standardize returns

In [ ]:
STANDARDIZE_RETURNS = False

y_model = y_clean
y_mean = None
y_std = None
if STANDARDIZE_RETURNS:
    y_model, y_mean, y_std = standardize(y_clean)
    print('Standardized returns with mean/std:', y_mean, y_std)
else:
    print('Using raw returns scale.')


## 8) Fit Model 1 (mandatory): mean switch

In [ ]:
model_m = build_switchpoint_mean_model(y_model)
idata_m = sample_model(
    model_m,
    draws=2000,
    tune=2000,
    chains=4,
    target_accept=0.9,
    random_seed=42,
)


### 8.1 Model 1 diagnostics

In [ ]:
summary_m = az.summary(idata_m, var_names=['tau','mu_1','mu_2','sigma'], round_to=6)
summary_m


In [ ]:
conv_m = compute_convergence_report(idata_m, ['tau','mu_1','mu_2','sigma'])
conv_m


In [ ]:
az.plot_trace(idata_m, var_names=['tau','mu_1','mu_2','sigma']);
plt.tight_layout();
plt.savefig(REPORTS_FIGURES_DIR / 'nb_task2_m1_trace.png', dpi=150, bbox_inches='tight');
plt.show();


In [ ]:
az.plot_posterior(idata_m, var_names=['tau']);
plt.tight_layout();
plt.savefig(REPORTS_FIGURES_DIR / 'nb_task2_m1_tau_posterior.png', dpi=150, bbox_inches='tight');
plt.show();


## 9) Fit Model 2 (extension): sigma switch

In [ ]:
model_s = build_switchpoint_sigma_model(y_model)
idata_s = sample_model(
    model_s,
    draws=2000,
    tune=2000,
    chains=4,
    target_accept=0.9,
    random_seed=42,
)


### 9.1 Model 2 diagnostics

In [ ]:
summary_s = az.summary(idata_s, var_names=['tau','mu','sigma_1','sigma_2'], round_to=6)
summary_s


In [ ]:
conv_s = compute_convergence_report(idata_s, ['tau','mu','sigma_1','sigma_2'])
conv_s


In [ ]:
az.plot_trace(idata_s, var_names=['tau','mu','sigma_1','sigma_2']);
plt.tight_layout();
plt.savefig(REPORTS_FIGURES_DIR / 'nb_task2_m2_trace.png', dpi=150, bbox_inches='tight');
plt.show();


In [ ]:
az.plot_posterior(idata_s, var_names=['tau']);
plt.tight_layout();
plt.savefig(REPORTS_FIGURES_DIR / 'nb_task2_m2_tau_posterior.png', dpi=150, bbox_inches='tight');
plt.show();


In [ ]:
az.plot_posterior(idata_s, var_names=['sigma_1','sigma_2']);
plt.tight_layout();
plt.savefig(REPORTS_FIGURES_DIR / 'nb_task2_m2_sigma_posterior.png', dpi=150, bbox_inches='tight');
plt.show();


## 10) Posterior predictive checks (PPC)

In [ ]:
with model_m:
    ppc_m = pm.sample_posterior_predictive(idata_m, random_seed=42)
idata_m.extend(ppc_m)
az.plot_ppc(idata_m, data_pairs={'obs':'obs'}, num_pp_samples=200);
plt.tight_layout();
plt.savefig(REPORTS_FIGURES_DIR / 'nb_task2_m1_ppc.png', dpi=150, bbox_inches='tight');
plt.show();


In [ ]:
with model_s:
    ppc_s = pm.sample_posterior_predictive(idata_s, random_seed=42)
idata_s.extend(ppc_s)
az.plot_ppc(idata_s, data_pairs={'obs':'obs'}, num_pp_samples=200);
plt.tight_layout();
plt.savefig(REPORTS_FIGURES_DIR / 'nb_task2_m2_ppc.png', dpi=150, bbox_inches='tight');
plt.show();


## 11) Impact summaries + tau sample mapping

In [ ]:
impact_mean = compute_impact_summary(idata_m, dates_clean)
impact_sigma = compute_sigma_impact_summary(idata_s, dates_clean)
impact_mean, impact_sigma


In [ ]:
pd.DataFrame([impact_mean.__dict__]).to_csv(REPORTS_INTERIM_DIR / 'nb_task2_m1_impact_summary.csv', index=False)
pd.DataFrame([impact_sigma]).to_csv(REPORTS_INTERIM_DIR / 'nb_task2_m2_sigma_impact_summary.csv', index=False)
print('Saved impact summaries to', REPORTS_INTERIM_DIR)


In [ ]:
tau_m_df = map_tau_samples_to_dates(idata_m, dates_clean)
tau_s_df = map_tau_samples_to_dates(idata_s, dates_clean)
tau_m_df.to_csv(REPORTS_INTERIM_DIR / 'nb_task2_m1_tau_samples.csv', index=False)
tau_s_df.to_csv(REPORTS_INTERIM_DIR / 'nb_task2_m2_tau_samples.csv', index=False)
tau_m_df.head()


### 11.1 Tau-date posterior mass plots

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 3))
ax.hist(pd.to_datetime(tau_m_df['tau_date']), bins=60)
ax.set_title('Posterior mass of change-point date (Model 1: mean switch)')
ax.set_xlabel('date')
plt.tight_layout()
plt.savefig(REPORTS_FIGURES_DIR / 'nb_task2_m1_tau_date_mass.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 3))
ax.hist(pd.to_datetime(tau_s_df['tau_date']), bins=60)
ax.set_title('Posterior mass of change-point date (Model 2: sigma switch)')
ax.set_xlabel('date')
plt.tight_layout()
plt.savefig(REPORTS_FIGURES_DIR / 'nb_task2_m2_tau_date_mass.png', dpi=150, bbox_inches='tight')
plt.show()


## 12) Optional model comparison (LOO)

In [ ]:
try:
    cmp = az.compare({'mean_switch': idata_m, 'sigma_switch': idata_s}, ic='loo')
    cmp.to_csv(REPORTS_INTERIM_DIR / 'nb_task2_model_comparison_loo.csv')
    cmp
except Exception as e:
    print('LOO comparison failed:', e)


## 13) Optional: events near inferred change point (±window)

In [ ]:
events_path = DATA_RAW_DIR / EVENTS_FILENAME
if events_path.exists():
    ev = load_events(events_path)

    # Model 1
    cp_mean = pd.to_datetime(impact_mean.tau_mode_date)
    ev_mean = ev.copy()
    ev_mean['days_from_cp'] = (ev_mean[COL_DATE] - cp_mean).dt.days
    ev_mean_near = ev_mean.loc[ev_mean['days_from_cp'].abs() <= EVENT_MATCH_WINDOW_DAYS].copy()
    ev_mean_near.to_csv(REPORTS_INTERIM_DIR / 'nb_task2_events_near_cp_mean_switch.csv', index=False)

    # Model 2
    cp_sigma = pd.to_datetime(impact_sigma['tau_mode_date'])
    ev_sigma = ev.copy()
    ev_sigma['days_from_cp'] = (ev_sigma[COL_DATE] - cp_sigma).dt.days
    ev_sigma_near = ev_sigma.loc[ev_sigma['days_from_cp'].abs() <= EVENT_MATCH_WINDOW_DAYS].copy()
    ev_sigma_near.to_csv(REPORTS_INTERIM_DIR / 'nb_task2_events_near_cp_sigma_switch.csv', index=False)

    print('Saved events-near-CP tables to', REPORTS_INTERIM_DIR)
    ev_mean_near.head()
else:
    print('No events file found; skipping.')


## 14) Save run metadata

In [ ]:
import json

meta = {
    'standardize_returns': STANDARDIZE_RETURNS,
    'y_mean_if_standardized': y_mean,
    'y_std_if_standardized': y_std,
    'priors_used': dict(prior_settings_summary(y_model, mu_prior_sigma=None, sigma_prior_sigma=None)),
    'm1_convergence': getattr(conv_m, '__dict__', str(conv_m)),
    'm2_convergence': getattr(conv_s, '__dict__', str(conv_s)),
    'm1_cp_date_mode': impact_mean.tau_mode_date,
    'm2_cp_date_mode': impact_sigma.get('tau_mode_date', None),
}
(REPORTS_INTERIM_DIR / 'nb_task2_run_metadata.json').write_text(json.dumps(meta, indent=2))
print('Saved:', REPORTS_INTERIM_DIR / 'nb_task2_run_metadata.json')
